In [2]:
# Load libraries 
## Raw Packages
import numpy as np
import pandas as pd
from math import pi
# from pandas_datareader import data as pdr
import datetime, time
from random import sample
## Data Source
import yfinance as yf
# To let pdr get data from yahoo -- https://stackoverflow.com/questions/74912452/typeerror-string-indices-must-be-integer-pandas-datareader
# yf.pdr_override()
from yahoo_fin import stock_info as si 
## Data Visualization 
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.layouts import column, row
# Site Scrapping
import requests
import json
import urllib.request
# Utilities
from IPython.display import clear_output

In [3]:
# ft100 = si.tickers_ftse100()
ft100= pd.read_html("https://en.wikipedia.org/wiki/FTSE_100_Index", attrs = {"id": "constituents"})[0].Ticker.unique()

ft100 = [i.replace(r'.', '-') for i in ft100]
# ft250 = si.tickers_ftse250()
ft250 = pd.read_html("https://en.wikipedia.org/wiki/FTSE_250_Index", attrs = {"id": "constituents"})[0].Ticker.unique()

ft250 = [i.replace(r'.', '-') for i in ft250]
ibo = si.tickers_ibovespa()
ibo = [i.replace(r'.', '-') for i in ibo]
n50 = si.tickers_nifty50()
n50 = [i.replace(r'.', '-') for i in n50]
nfb = si.tickers_niftybank()
nfb = [i.replace(r'.', '-') for i in nfb]
nas = si.tickers_nasdaq()
nas = [i.replace(r'.', '-') for i in nas]
dow = si.tickers_dow()
dow = [i.replace(r'.', '-') for i in dow]
othr = si.tickers_other()
othr = [i.replace(r'.', '-') for i in othr]
sp500 = si.tickers_sp500()
sp500 = [i.replace(r'.', '-') for i in sp500]


all_yh_tic = {'ft100':ft100, 'ft250': ft250, 'ibo':ibo, 'n50':n50, 'nfb':nfb, 'nas':nas, 'dow':dow, 'other':othr, 'sp500': sp500}
for key, value in all_yh_tic.items():
    print(key, '\t', len(value))
    
distinct_tickers = list(set(sp500) | set(dow)| set(nas)|set(othr)|set(ft100)| set(ft250)|set(ibo)|set(n50)|set(nfb))
print('Total Distinct Tickers',len(distinct_tickers))

/opt/anaconda3/envs/p312/lib/python3.12/site-packages/yahoo_fin/stock_info.py:235: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  table = pd.read_html(requests.get(site, headers=headers).text)[0]


ft100 	 100
ft250 	 249
ibo 	 64
n50 	 29
nfb 	 12
nas 	 4808
dow 	 30
other 	 6342
sp500 	 503
Total Distinct Tickers 11511


In [41]:
import pandas as pd
from yahooquery import Ticker
import time
tickers = Ticker(['ALLY', 'GOOGL'], asynchronous=True)
datasi = tickers.get_modules("summaryProfile quoteType")
dfsi = pd.DataFrame.from_dict(datasi).T
dataframes = [pd.json_normalize([x for x in dfsi[module] if isinstance(x, dict)]) for module in ['summaryProfile', 'quoteType']]

In [54]:
sp = pd.json_normalize(dfsi['summaryProfile'])
qt = pd.json_normalize(dfsi['quoteType'])
pd.concat([qt, sp], axis = 1)

,exchange,quoteType,symbol,underlyingSymbol,shortName,longName,firstTradeDateEpochUtc,timeZoneFullName,timeZoneShortName,uuid,...,country,phone,website,industry,sector,longBusinessSummary,fullTimeEmployees,companyOfficers,maxAge,address2
0,NMS,EQUITY,GOOGL,GOOGL,Alphabet Inc.,Alphabet Inc.,2004-08-19 08:30:00,America/New_York,EST,e15ce71f-f533-3912-9f11-a46c09e2412b,...,United States,650 253 0000,https://abc.xyz,Internet Content & Information,Communication Services,Alphabet Inc. offers various products and plat...,181269,[],86400,NaN
1,NYQ,EQUITY,ALLY,ALLY,Ally Financial Inc.,Ally Financial Inc.,2014-01-28 08:30:00,America/New_York,EST,8a909f5e-e918-3002-951c-d3779f9fbb0b,...,United States,866 710 4623,https://www.ally.com,Credit Services,Financial Services,"Ally Financial Inc., a digital financial-servi...",11100,[],86400,Floor 10 500 Woodward Avenue


In [57]:
import pandas as pd
from yahooquery import Ticker
import time
ticker_list = distinct_tickers
sector_df = pd.DataFrame()
if len(ticker_list) > 1000:
    symbols_a = np.array_split(ticker_list, 30)
else: 
    symbols_a =[ticker_list]
for i, symb_list in enumerate(symbols_a):
    if i%5 == 0:
        clear_output()
    tickers = Ticker(symb_list, asynchronous=True)

    datasi = tickers.get_modules("summaryProfile quoteType")
    dfsi = pd.DataFrame.from_dict(datasi).T
    sp = pd.json_normalize(dfsi['summaryProfile'])
    qt = pd.json_normalize(dfsi['quoteType'])
    
    dfsi = pd.concat([qt, sp], axis = 1)
    if 'website' not in dfsi.columns:
        dfsi.loc[:,'website'] = ""
    if 'country' not in dfsi.columns:
        dfsi.loc[:,'country'] = ""

    sector_df = pd.concat([sector_df, dfsi[['symbol','country','website', 'industry', 'sector']]]).reset_index(drop = True)
    time.sleep(1)
    print(i)


25
26
27
28
29


In [60]:
sector_df[sector_df.symbol=='ALLY']

,symbol,country,website,industry,sector
312,ALLY,United States,https://www.ally.com,Credit Services,Financial Services


In [61]:
sector_df1 = sector_df.dropna(how = 'all', subset = ['country', 'website', 'industry', 'sector'])
sector_df1.columns = [col.upper() for col in sector_df1.columns]


sector_df1.to_csv('./stock_analysis/static/ticker_sectors.csv', index = False)

In [ ]:
dfsi